In [1]:
import json
import requests
from src import utils
import time
import os

In [2]:
api_path = '/userhomes/philhoon/kt-ai-challenge/api.json'

In [3]:
api = utils.read_json_file(api_path)

In [4]:
input_file = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl'
# in_path = '/userhomes/philhoon/kt-ai-challenge/data/openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-train.jsonl'

In [5]:
output_path = '/userhomes/philhoon/kt-ai-challenge/result'

In [6]:
os.makedirs(output_path, exist_ok=True)
output_file_name = 'result-' + input_file.split('/')[-1]
output_file = output_path + '/' + output_file_name
print(f'output_file : {output_file}')

output_file : /userhomes/philhoon/kt-ai-challenge/result/result-openai-gpt-3.5-turbo-0301-2048-0.0-KOR-gsm8k-test.jsonl


In [7]:
jsonl_writer = utils.JSONLWriter(output_file)

In [8]:
data = utils.read_jsonlines(input_file)

In [9]:
# file_name

In [10]:
len(data)

1319

In [11]:
# api

In [12]:
# url = api['url']

In [13]:
# headers = api['headers']

In [14]:
# question = "질문 : 파운드 스털링은 가치가 있나요? 용어설명 :  사실목록 : ['1파운드는 법정 화폐입니다.', '법정 화폐는 정부 법령에 의해 뒷받침되며 내재적 가치가 없습니다', '2020년 5월 기준 1파운드 스털링은 약 1.24달러의 가치가 있습니다.']. 사실목록을 바탕으로 질문에 대한 답변을 사실 또는 거짓으로 대답하세요"/

In [15]:
# headers

In [16]:
class model_api():
    def __init__(self, url, headers):
        self.url = url
        self.headers = headers
        
    def generate(self, body):
        
        try:
            response = requests.post(self.url, data=body, headers=self.headers, verify=False)
        
        except:
            print("retrying due to an error......")
            time.sleep(20)
            return self.generate(body)
        
        return response
        
        

In [17]:
model = model_api(api['url'], api['headers'])

In [18]:
BODY_TEMP = {
    "serviceInstanceId" : "ddm3kz9g", 
    "nluType" : "009",
    "apiType" : "002", 
    "utterance" : None
}

In [19]:
# body = json.dumps(body_temp)

In [20]:
# print(body)

In [21]:
# response = api.generate(body)

In [22]:
# context = response.json()['data'][0]['result'][0]

In [23]:
# print(context)

In [24]:
def get_gsm8k_zs(que):
    prompt = f'질문 : {que} \n정답 : '
    return prompt

In [25]:
def get_body(prompt, body_temp):
    body_temp["utterance"] = prompt
    return json.dumps(body_temp)

In [26]:
# from pprint import pprint

### GSM8K-TEST
    26 out of 1319 : contains the answer in the question due to translation
### GSM8K-TRAIN
    163 out of 7473 : contains the answer in the question due to translation

In [27]:
cnt = 0
split_cnt = 0
for ins in data:
    kor_que = ins['kor_question']
    
    # checking whether there is an answer 
    test = kor_que.split('\n')
    
    if len(test[0]) != len(kor_que):
        split_cnt += 1
        continue

    kor_ans = ins['kor_answer']
    prompt = get_gsm8k_zs(kor_que)
    
    body = get_body(prompt, BODY_TEMP)
    response = model.generate(body)
    res_context = response.json()['data'][0]['result'][0]
    
    print(f'prompt : ')
    print(prompt)
    print(f'res_context : ')
    print(res_context)
    print('----')
    
    ins['kor_infer'] = res_context
#     jsonl_writer.write_json_line(ins)
    
#     print(prompt)
    if cnt == 5:
        break
    else:
        cnt += 1

/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 한 벌의 로브를 만드는 데 파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요합니다. 총 몇 덩이의 섬유가 필요한가요? 
정답 : 
res_context : 
파란색 섬유 2 덩이와 그 절반만큼의 흰색 섬유가 필요하므로 총 4 덩이의 섬유가 필요합니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 조쉬는 집을 뒤집어 판매해보기로 결정합니다. 그는 $80,000에 집을 구매하고 $50,000을 수리합니다. 이로 인해 집의 가치가 150% 증가합니다. 그가 얼마의 이익을 얻었습니까? 
정답 : 
res_context : 
조쉬는 집을 팔아 $150,000의 이익을 얻었습니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


...

prompt : 
질문 : 매일, 웬디는 각 닭에게 씨앗, 멸치, 채소가 섞인 닭 사료 3컵씩을 주어 건강하게 유지합니다. 그녀는 닭에게 3번의 식사로 사료를 제공합니다. 아침에는 15컵의 사료를 주고, 오후에는 25컵의 사료를 또 다시 주며, 만약 웬디의 닭 떼가 20마리라면, 하루의 마지막 식사에서 닭에게 몇 컵의 사료를 주어야 할까요? 
정답 : 
res_context : 
웬디는 닭에게 3번의 식사를 제공하므로, 하루에 3 \* 3 = 15 (3컵)를 주게 됩니다. 닭 떼가 20마리라면, 한 끼 식사에 20마리의 닭을 사용하게 됩니다. 따라서 웬디는 닭에게 15 \* 20 = 150 (15컵)을 주게 됩니다. 마지막으로 3 \* 3 = 30 (3컵)을 제공하게 됩니다. 따라서 닭에게 15 \* 30 = 180 (3컵)을 주게 됩니다. 따라서 웬디의 닭 떼가 20마리라면, 하루의 마지막 식사에서 닭에게 180 (3컵)의 사료를 주어야 합니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 카일러는 새 아파트를 위해 유리잔을 사러 가게에 갔습니다. 하나의 유리잔 가격은 $5이지만, 두 번째 유리잔부터는 가격의 60%만 지불하면 됩니다. 카일러는 16개의 유리잔을 사고 싶습니다. 그들을 위해 얼마를 지불해야 할까요? 
정답 : 
res_context : 
정답은 $16입니다.
----


/userhomes/philhoon/miniconda3/envs/self-debate/lib/python3.8/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'aiapi.genielabs.ai'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


prompt : 
질문 : 툴루즈는 찰스턴의 양의 두 배를 가지고 있습니다. 찰스턴은 시애틀의 양의 4 배를 가지고 있습니다. 시애틀은 양이 20마리 있으므로, 툴루즈, 찰스턴 및 시애틀이 함께 가지고 있는 양은 얼마입니까? 
정답 : 
res_context : 
툴루즈, 찰스턴, 시애틀이 함께 가지고 있는 양은 4 + 2 + 4 = 10마리입니다.
----
